In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import pickle
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.ensemble import AdaBoostRegressor
from sklearn import model_selection as ms
from scipy.stats import pearsonr
from sklearn.metrics import make_scorer
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold

rand_st=4

In [2]:
output=pd.read_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx')
errors=pd.read_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx')

In [3]:
elos_novice=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\elos_novice.csv')
elos_avg=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\elos_avg.csv')
elos_master=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\elos_master.csv')
elos_grandmaster=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\elos_grandmaster.csv')
features_novice=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features_novice.csv')
features_master=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features_master.csv')
features_avg=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features_avg.csv')
features_grandmaster=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features_grandmaster.csv')

In [42]:
X_train_mean_novice, X_test_mean_novice, Y_train_mean_novice, Y_test_mean_novice =ms.train_test_split(features_novice, elos_novice['MeanElos'], test_size = 0.3,random_state=rand_st)
X_train_diff_novice, X_test_diff_novice, Y_train_diff_novice, Y_test_diff_novice =ms.train_test_split(features_novice, elos_novice['DiffElos'], test_size = 0.3,random_state=rand_st)

X_train_mean_avg, X_test_mean_avg, Y_train_mean_avg, Y_test_mean_avg =ms.train_test_split(features_avg, elos_avg['MeanElos'], test_size = 0.3,random_state=rand_st)
X_train_diff_avg, X_test_diff_avg, Y_train_diff_avg, Y_test_diff_avg =ms.train_test_split(features_avg, elos_avg['DiffElos'], test_size = 0.3,random_state=rand_st)

X_train_mean_master, X_test_mean_master, Y_train_mean_master, Y_test_mean_master =ms.train_test_split(features_master, elos_master['MeanElos'], test_size = 0.3,random_state=rand_st)
X_train_diff_master, X_test_diff_master, Y_train_diff_master, Y_test_diff_master =ms.train_test_split(features_master, elos_master['DiffElos'], test_size = 0.3,random_state=rand_st)

X_train_mean_grandmaster, X_test_mean_grandmaster, Y_train_mean_grandmaster, Y_test_mean_grandmaster =ms.train_test_split(features_grandmaster, elos_grandmaster['MeanElos'], test_size = 0.3,random_state=rand_st)
X_train_diff_grandmaster, X_test_diff_grandmaster, Y_train_diff_grandmaster, Y_test_diff_grandmaster =ms.train_test_split(features_grandmaster, elos_grandmaster['DiffElos'], test_size = 0.3,random_state=rand_st)

In [5]:
model_mean=AdaBoostRegressor(base_estimator=None, learning_rate=2, loss='square',n_estimators=500, random_state=4)
model_diff=AdaBoostRegressor(base_estimator=None, learning_rate=2, loss='square',n_estimators=50, random_state=4)

In [6]:
def find_mean_error(pred,elos):
    er=0
    count=0
    for ind, val in enumerate(pred):
        er+=abs(elos.iloc[ind,0]-val[0])
        er+=abs(elos.iloc[ind,1]-val[1])
        count+=2
    return round(er/count,3)

def find_abs_error(pred,elos):
    er=0
    for ind, val in enumerate(pred):
        er+=abs(elos.iloc[ind,0]-val[0])
        er+=abs(elos.iloc[ind,1]-val[1])
    return round(er,3)

def find_root_mean_squared_error(pred,elos):
    er=0
    count=0
    for ind, val in enumerate(pred):
        er+=(elos.iloc[ind,0]-val[0])**2
        er+=(elos.iloc[ind,1]-val[1])**2
        count+=2
    return round((er/count)**0.5,3)

In [7]:
def predictions_report(pred_target,predictions):
    mae_report=mae(pred_target,predictions)
    print ('MAE = {:.3f}'.format(mae_report))
    
    rmse_report = (mse(pred_target, predictions))**0.5
    print ('RMSE = {:.3f}'.format(rmse_report))
    
    corr_coef = pearsonr(pred_target,predictions)
    print("Correlation coefficient = {:.3f}".format(corr_coef[0]))
    
    return (mae_report,rmse_report,corr_coef)

### Подбор модели для новичков

In [8]:
start1 = time.time()

ada1=model_mean.fit(X_train_mean_novice,Y_train_mean_novice)
ada_pred1=ada1.predict(X_test_mean_novice)
scores1=predictions_report(Y_test_mean_novice,ada_pred1)
full_time1=round(time.time() - start1,3)
print(full_time1, 'c')

MAE = 233.165
RMSE = 286.956
Correlation coefficient = 0.098
47.529 c


In [9]:
output = output.append(pd.Series([str(model_mean),
                                 scores1[0],scores1[1], 
                                 scores1[2][0],full_time1,'Обучение на среднем ЭЛО по начинающим'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",185.454373,232.791414,0.272042,0.104,Обучение на среднем ЭЛО по начинающим
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",167.847740,213.396908,0.604164,0.117,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",148.774801,195.120413,0.195972,0.104,Обучение на среднем ЭЛО по средним игрокам
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",153.680899,194.776371,0.621906,0.105,Обучение на разнице в ЭЛО по средним игрокам
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",144.547821,186.916248,0.211849,0.113,Обучение на среднем ЭЛО по мастерам
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",137.465903,174.230219,0.542416,0.120,Обучение на разнице в ЭЛО по мастерам
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",132.400047,179.033987,0.462350,0.026,Обучение на среднем ЭЛО по гроссмейстерам
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",91.863665,120.573687,0.523059,0.030,Обучение на разнице в ЭЛО по гроссмейстерам
8,"AdaBoostRegressor(base_estimator=None, learnin...",233.165385,286.956260,0.098477,47.529,Обучение на среднем ЭЛО по начинающим


In [10]:
start2 = time.time()

ada2=model_diff.fit(X_train_diff_novice,Y_train_diff_novice)
ada_pred2=ada2.predict(X_test_mean_novice)
scores2=predictions_report(Y_test_diff_novice,ada_pred2)
full_time2=round(time.time() - start2,3)
print(full_time2, 'c')

MAE = 180.602
RMSE = 225.786
Correlation coefficient = 0.521
5.654 c


In [11]:
output = output.append(pd.Series([str(model_diff),
                                 scores2[0],scores2[1], 
                                 scores2[2][0],full_time2,'Обучение на разнице ЭЛО по начинающим'],
                                 index=output.columns ),
                      ignore_index=True)


output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",185.454373,232.791414,0.272042,0.104,Обучение на среднем ЭЛО по начинающим
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",167.847740,213.396908,0.604164,0.117,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",148.774801,195.120413,0.195972,0.104,Обучение на среднем ЭЛО по средним игрокам
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",153.680899,194.776371,0.621906,0.105,Обучение на разнице в ЭЛО по средним игрокам
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",144.547821,186.916248,0.211849,0.113,Обучение на среднем ЭЛО по мастерам
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",137.465903,174.230219,0.542416,0.120,Обучение на разнице в ЭЛО по мастерам
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",132.400047,179.033987,0.462350,0.026,Обучение на среднем ЭЛО по гроссмейстерам
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",91.863665,120.573687,0.523059,0.030,Обучение на разнице в ЭЛО по гроссмейстерам
8,"AdaBoostRegressor(base_estimator=None, learnin...",233.165385,286.956260,0.098477,47.529,Обучение на среднем ЭЛО по начинающим
9,"AdaBoostRegressor(base_estimator=None, learnin...",180.602289,225.785633,0.520742,5.654,Обучение на разнице ЭЛО по начинающим


In [12]:
output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)

In [13]:
white_elos_novice = ada_pred1 + ada_pred2/2
black_elos_novice = ada_pred1-ada_pred2/2
pred_elos1=list(zip(white_elos_novice,black_elos_novice))
print(len(pred_elos1))
pred_elos1[:5]

1935


[(2377.8093138486774, 2313.740960374357),
 (2426.951836622807, 2397.19920504386),
 (2308.8919734186593, 2356.4441348019445),
 (1877.498167707006, 1817.687603043982),
 (2157.0698663560916, 2044.8876313990713)]

In [14]:
find_abs_error(pred_elos1,elos_novice),find_mean_error(pred_elos1,elos_novice),find_root_mean_squared_error(pred_elos1,elos_novice)

(1053871.466, 272.318, 339.773)

In [15]:
inner_mae_novice=find_mean_error(pred_elos1,elos_novice)

In [17]:
errors =errors.append(pd.Series(['Слабые игроки','AdaBoost',
                  find_abs_error(pred_elos1,elos_novice),
                  find_mean_error(pred_elos1,elos_novice),
                   find_root_mean_squared_error(pred_elos1,elos_novice) ],
                                   index=errors.columns ),
                      ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

,Группа,Модель,Абсолютная ошибка,MAE,RMSE
0,Слабые игроки,LinearRegression,2.636124e+06,204.41400,256.08000
1,Средние игроки,LinearRegression,2.422902e+06,171.06100,218.08000
2,Мастера,LinearRegression,3.121464e+06,161.26600,206.22300
3,Гроссмейстеры,LinearRegression,2.933576e+05,140.09400,188.90800
4,Общая ошибка,LinearRegression,2.118462e+06,169.20875,217.32275
5,Слабые игроки,AdaBoost,1.053871e+06,272.31800,339.77300


In [18]:
errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)

### Подбор модели для среднего уровня

In [19]:
start3 = time.time()

ada3=model_mean.fit(X_train_mean_avg,Y_train_mean_avg)
ada_pred3=ada3.predict(X_test_mean_avg)
scores3=predictions_report(Y_test_mean_avg,ada_pred3)
full_time3=round(time.time() - start3,3)
print(full_time3, 'c')

MAE = 178.714
RMSE = 223.999
Correlation coefficient = 0.087
52.226 c


In [21]:
output = output.append(pd.Series([str(model_mean),
                                 scores3[0],scores3[1], 
                                 scores3[2][0],full_time3,'Обучение на среднем ЭЛО по начинающим'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",185.454373,232.791414,0.272042,0.104,Обучение на среднем ЭЛО по начинающим
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",167.847740,213.396908,0.604164,0.117,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",148.774801,195.120413,0.195972,0.104,Обучение на среднем ЭЛО по средним игрокам
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",153.680899,194.776371,0.621906,0.105,Обучение на разнице в ЭЛО по средним игрокам
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",144.547821,186.916248,0.211849,0.113,Обучение на среднем ЭЛО по мастерам
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",137.465903,174.230219,0.542416,0.120,Обучение на разнице в ЭЛО по мастерам
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",132.400047,179.033987,0.462350,0.026,Обучение на среднем ЭЛО по гроссмейстерам
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",91.863665,120.573687,0.523059,0.030,Обучение на разнице в ЭЛО по гроссмейстерам
8,"AdaBoostRegressor(base_estimator=None, learnin...",233.165385,286.956260,0.098477,47.529,Обучение на среднем ЭЛО по начинающим
9,"AdaBoostRegressor(base_estimator=None, learnin...",180.602289,225.785633,0.520742,5.654,Обучение на разнице ЭЛО по начинающим


In [24]:
start4 = time.time()

ada4=model_diff.fit(X_train_diff_avg,Y_train_diff_avg)
ada_pred4=ada4.predict(X_test_mean_avg)
scores4=predictions_report(Y_test_diff_avg,ada_pred4)
full_time4=round(time.time() - start4,3)
print(full_time4, 'c')

MAE = 173.179
RMSE = 217.897
Correlation coefficient = 0.534
6.853 c


In [25]:
output = output.append(pd.Series([str(model_diff),
                                 scores4[0],scores4[1], 
                                 scores4[2][0],full_time4,'Обучение на разнице ЭЛО по средним игрокам'],
                                 index=output.columns ),
                      ignore_index=True)


output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",185.454373,232.791414,0.272042,0.104,Обучение на среднем ЭЛО по начинающим
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",167.847740,213.396908,0.604164,0.117,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",148.774801,195.120413,0.195972,0.104,Обучение на среднем ЭЛО по средним игрокам
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",153.680899,194.776371,0.621906,0.105,Обучение на разнице в ЭЛО по средним игрокам
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",144.547821,186.916248,0.211849,0.113,Обучение на среднем ЭЛО по мастерам
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",137.465903,174.230219,0.542416,0.120,Обучение на разнице в ЭЛО по мастерам
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",132.400047,179.033987,0.462350,0.026,Обучение на среднем ЭЛО по гроссмейстерам
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",91.863665,120.573687,0.523059,0.030,Обучение на разнице в ЭЛО по гроссмейстерам
8,"AdaBoostRegressor(base_estimator=None, learnin...",233.165385,286.956260,0.098477,47.529,Обучение на среднем ЭЛО по начинающим
9,"AdaBoostRegressor(base_estimator=None, learnin...",180.602289,225.785633,0.520742,5.654,Обучение на разнице ЭЛО по начинающим


In [26]:
output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)

In [27]:
white_elos_avg = ada_pred3 + ada_pred3/2
black_elos_avg = ada_pred3-ada_pred4/2
pred_elos2=list(zip(white_elos_avg,black_elos_avg))
print(len(pred_elos2))
pred_elos2[:5]

2125


[(3516.738681493249, 2434.8345697134478),
 (3454.8575342465756, 2393.580471548999),
 (3609.25881147541, 2470.6351408383634),
 (3676.6056469128143, 2519.4669684373716),
 (3372.349156626506, 2398.109515270384)]

In [28]:
find_abs_error(pred_elos2,elos_avg),find_mean_error(pred_elos2,elos_avg),find_root_mean_squared_error(pred_elos2,elos_avg)

(3092972.694, 727.758, 912.937)

In [29]:
inner_mae_avg=find_mean_error(pred_elos2,elos_avg)

In [30]:
errors =errors.append(pd.Series(['Средние игроки','AdaBoost',
                        find_abs_error(pred_elos2,elos_avg),
                        find_mean_error(pred_elos2,elos_avg),
                        find_root_mean_squared_error(pred_elos2,elos_avg)],
                                index=errors.columns ),
                      ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

,Группа,Модель,Абсолютная ошибка,MAE,RMSE
0,Слабые игроки,LinearRegression,2.636124e+06,204.41400,256.08000
1,Средние игроки,LinearRegression,2.422902e+06,171.06100,218.08000
2,Мастера,LinearRegression,3.121464e+06,161.26600,206.22300
3,Гроссмейстеры,LinearRegression,2.933576e+05,140.09400,188.90800
4,Общая ошибка,LinearRegression,2.118462e+06,169.20875,217.32275
5,Слабые игроки,AdaBoost,1.053871e+06,272.31800,339.77300
6,Средние игроки,AdaBoost,3.092973e+06,727.75800,912.93700


### Подбор модели для мастеров

In [31]:
start5 = time.time()

ada5=model_mean.fit(X_train_mean_master,Y_train_mean_master)
ada_pred5=ada5.predict(X_test_mean_master)
scores5=predictions_report(Y_test_mean_master,ada_pred5)
full_time5=round(time.time() - start5,3)
print(full_time5, 'c')

MAE = 156.457
RMSE = 202.064
Correlation coefficient = 0.101
70.938 c


In [32]:
output = output.append(pd.Series([str(model_mean),
                                 scores5[0],scores5[1], 
                                 scores5[2][0],full_time5,'Обучение на среднем ЭЛО по мастерам'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",185.454373,232.791414,0.272042,0.104,Обучение на среднем ЭЛО по начинающим
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",167.847740,213.396908,0.604164,0.117,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",148.774801,195.120413,0.195972,0.104,Обучение на среднем ЭЛО по средним игрокам
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",153.680899,194.776371,0.621906,0.105,Обучение на разнице в ЭЛО по средним игрокам
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",144.547821,186.916248,0.211849,0.113,Обучение на среднем ЭЛО по мастерам
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",137.465903,174.230219,0.542416,0.120,Обучение на разнице в ЭЛО по мастерам
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",132.400047,179.033987,0.462350,0.026,Обучение на среднем ЭЛО по гроссмейстерам
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",91.863665,120.573687,0.523059,0.030,Обучение на разнице в ЭЛО по гроссмейстерам
8,"AdaBoostRegressor(base_estimator=None, learnin...",233.165385,286.956260,0.098477,47.529,Обучение на среднем ЭЛО по начинающим
9,"AdaBoostRegressor(base_estimator=None, learnin...",180.602289,225.785633,0.520742,5.654,Обучение на разнице ЭЛО по начинающим


In [33]:
start6 = time.time()

ada6=model_diff.fit(X_train_diff_master,Y_train_diff_master)
ada_pred6=ada6.predict(X_test_mean_master)
scores6=predictions_report(Y_test_diff_master,ada_pred6)
full_time6=round(time.time() - start6,3)
print(full_time6, 'c')

MAE = 149.616
RMSE = 187.697
Correlation coefficient = 0.479
8.024 c


In [35]:
output = output.append(pd.Series([str(model_diff),
                                 scores6[0],scores6[1], 
                                 scores6[2][0],full_time6,'Обучение на разнице ЭЛО по мастерам'],
                                 index=output.columns ),
                      ignore_index=True)


output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",185.454373,232.791414,0.272042,0.104,Обучение на среднем ЭЛО по начинающим
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",167.847740,213.396908,0.604164,0.117,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",148.774801,195.120413,0.195972,0.104,Обучение на среднем ЭЛО по средним игрокам
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",153.680899,194.776371,0.621906,0.105,Обучение на разнице в ЭЛО по средним игрокам
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",144.547821,186.916248,0.211849,0.113,Обучение на среднем ЭЛО по мастерам
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",137.465903,174.230219,0.542416,0.120,Обучение на разнице в ЭЛО по мастерам
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",132.400047,179.033987,0.462350,0.026,Обучение на среднем ЭЛО по гроссмейстерам
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",91.863665,120.573687,0.523059,0.030,Обучение на разнице в ЭЛО по гроссмейстерам
8,"AdaBoostRegressor(base_estimator=None, learnin...",233.165385,286.956260,0.098477,47.529,Обучение на среднем ЭЛО по начинающим
9,"AdaBoostRegressor(base_estimator=None, learnin...",180.602289,225.785633,0.520742,5.654,Обучение на разнице ЭЛО по начинающим


In [36]:
output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)

In [37]:
white_elos_master = ada_pred5 + ada_pred6/2
black_elos_master = ada_pred5-ada_pred6/2
pred_elos3=list(zip(white_elos_master,black_elos_master))
print(len(pred_elos3))
pred_elos3[:5]

2904


[(2455.000336249679, 2120.728783389147),
 (2379.542249556961, 2552.0693734054767),
 (2202.4407072966683, 2183.2558444274696),
 (2320.0466376390905, 2590.236381228834),
 (2350.9950409022977, 2317.015968272014)]

In [38]:
find_abs_error(pred_elos3,elos_master),find_mean_error(pred_elos3,elos_master),find_root_mean_squared_error(pred_elos3,elos_master)

(1130376.487, 194.624, 246.273)

In [39]:
inner_mae_master=find_mean_error(pred_elos3,elos_master)

In [40]:
errors =errors.append(pd.Series(['Мастера','AdaBoost',
                                 find_abs_error(pred_elos3,elos_master),
                                 find_mean_error(pred_elos3,elos_master),
                                 find_root_mean_squared_error(pred_elos3,elos_master)],
                                index=errors.columns ),
                      ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

,Группа,Модель,Абсолютная ошибка,MAE,RMSE
0,Слабые игроки,LinearRegression,2.636124e+06,204.41400,256.08000
1,Средние игроки,LinearRegression,2.422902e+06,171.06100,218.08000
2,Мастера,LinearRegression,3.121464e+06,161.26600,206.22300
3,Гроссмейстеры,LinearRegression,2.933576e+05,140.09400,188.90800
4,Общая ошибка,LinearRegression,2.118462e+06,169.20875,217.32275
5,Слабые игроки,AdaBoost,1.053871e+06,272.31800,339.77300
6,Средние игроки,AdaBoost,3.092973e+06,727.75800,912.93700
7,Мастера,AdaBoost,1.130376e+06,194.62400,246.27300


### Подбор модели для гроссмейстеров

In [43]:
start7 = time.time()

ada7=model_mean.fit(X_train_mean_grandmaster,Y_train_mean_grandmaster)
ada_pred7=ada7.predict(X_test_mean_grandmaster)
scores7=predictions_report(Y_test_mean_grandmaster,ada_pred7)
full_time7=round(time.time() - start7,3)
print(full_time7, 'c')

MAE = 166.669
RMSE = 240.324
Correlation coefficient = 0.109
4.31 c


In [45]:
output = output.append(pd.Series([str(model_mean),
                                 scores7[0],scores7[1], 
                                 scores7[2][0],full_time7,'Обучение на среднем ЭЛО по гроссмейстерам'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",185.454373,232.791414,0.272042,0.104,Обучение на среднем ЭЛО по начинающим
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",167.847740,213.396908,0.604164,0.117,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",148.774801,195.120413,0.195972,0.104,Обучение на среднем ЭЛО по средним игрокам
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",153.680899,194.776371,0.621906,0.105,Обучение на разнице в ЭЛО по средним игрокам
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",144.547821,186.916248,0.211849,0.113,Обучение на среднем ЭЛО по мастерам
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",137.465903,174.230219,0.542416,0.120,Обучение на разнице в ЭЛО по мастерам
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",132.400047,179.033987,0.462350,0.026,Обучение на среднем ЭЛО по гроссмейстерам
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",91.863665,120.573687,0.523059,0.030,Обучение на разнице в ЭЛО по гроссмейстерам
8,"AdaBoostRegressor(base_estimator=None, learnin...",233.165385,286.956260,0.098477,47.529,Обучение на среднем ЭЛО по начинающим
9,"AdaBoostRegressor(base_estimator=None, learnin...",180.602289,225.785633,0.520742,5.654,Обучение на разнице ЭЛО по начинающим


In [46]:
start8 = time.time()

ada8=model_diff.fit(X_train_diff_grandmaster,Y_train_diff_grandmaster)
ada_pred8=ada8.predict(X_test_mean_grandmaster)
scores8=predictions_report(Y_test_diff_grandmaster,ada_pred8)
full_time8=round(time.time() - start8,3)
print(full_time8, 'c')

MAE = 192.125
RMSE = 226.849
Correlation coefficient = 0.121
0.544 c


In [47]:
output = output.append(pd.Series([str(model_diff),
                                 scores8[0],scores8[1], 
                                 scores8[2][0],full_time8,'Обучение на разнице ЭЛО по гроссмейстерам'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",185.454373,232.791414,0.272042,0.104,Обучение на среднем ЭЛО по начинающим
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",167.847740,213.396908,0.604164,0.117,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",148.774801,195.120413,0.195972,0.104,Обучение на среднем ЭЛО по средним игрокам
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",153.680899,194.776371,0.621906,0.105,Обучение на разнице в ЭЛО по средним игрокам
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",144.547821,186.916248,0.211849,0.113,Обучение на среднем ЭЛО по мастерам
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",137.465903,174.230219,0.542416,0.120,Обучение на разнице в ЭЛО по мастерам
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",132.400047,179.033987,0.462350,0.026,Обучение на среднем ЭЛО по гроссмейстерам
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",91.863665,120.573687,0.523059,0.030,Обучение на разнице в ЭЛО по гроссмейстерам
8,"AdaBoostRegressor(base_estimator=None, learnin...",233.165385,286.956260,0.098477,47.529,Обучение на среднем ЭЛО по начинающим
9,"AdaBoostRegressor(base_estimator=None, learnin...",180.602289,225.785633,0.520742,5.654,Обучение на разнице ЭЛО по начинающим


In [48]:
white_elos_grandmaster = ada_pred7 + ada_pred8/2
black_elos_grandmaster = ada_pred7-ada_pred8/2
pred_elos4=list(zip(white_elos_grandmaster,black_elos_grandmaster))
print(len(pred_elos4))
pred_elos4[:5]

315


[(2704.8043290043292, 2671.870995670996),
 (2752.9594496302584, 2554.863559219299),
 (2726.8947368421054, 2550.8947368421054),
 (2784.3020833333335, 2527.3020833333335),
 (2568.791818181818, 2663.571818181818)]

In [49]:
find_abs_error(pred_elos4,elos_grandmaster),find_mean_error(pred_elos4,elos_grandmaster),find_root_mean_squared_error(pred_elos4,elos_grandmaster)

(138186.331, 219.343, 297.876)

In [50]:
inner_mae_grandmaster=find_mean_error(pred_elos4,elos_grandmaster)

In [51]:
errors =errors.append(pd.Series(['Гроссместеры','AdaBoost',find_abs_error(pred_elos4,elos_grandmaster),
                                 find_mean_error(pred_elos4,elos_grandmaster),
                                 find_root_mean_squared_error(pred_elos4,elos_grandmaster)],
                                index=errors.columns ),
                      ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

,Группа,Модель,Абсолютная ошибка,MAE,RMSE
0,Слабые игроки,LinearRegression,2.636124e+06,204.41400,256.08000
1,Средние игроки,LinearRegression,2.422902e+06,171.06100,218.08000
2,Мастера,LinearRegression,3.121464e+06,161.26600,206.22300
3,Гроссмейстеры,LinearRegression,2.933576e+05,140.09400,188.90800
4,Общая ошибка,LinearRegression,2.118462e+06,169.20875,217.32275
5,Слабые игроки,AdaBoost,1.053871e+06,272.31800,339.77300
6,Средние игроки,AdaBoost,3.092973e+06,727.75800,912.93700
7,Мастера,AdaBoost,1.130376e+06,194.62400,246.27300
8,Гроссместеры,AdaBoost,1.381863e+05,219.34300,297.87600


In [52]:
ada_mae=(inner_mae_grandmaster+inner_mae_master+inner_mae_avg+inner_mae_novice)/4
ada_mae

353.51075

In [53]:
ada_abs=(find_abs_error(pred_elos1,elos_novice)+
        find_abs_error(pred_elos2,elos_avg)+
        find_abs_error(pred_elos3,elos_master)+
        find_abs_error(pred_elos4,elos_grandmaster))/4
ada_abs

1353851.7445

In [54]:
ada_rmse=(find_root_mean_squared_error(pred_elos1,elos_novice)+
         find_root_mean_squared_error(pred_elos2,elos_avg)+
         find_root_mean_squared_error(pred_elos3,elos_master)+
         find_root_mean_squared_error(pred_elos4,elos_grandmaster))/4
ada_rmse

449.21475

In [55]:
errors = errors.append(pd.Series(['Общая ошибка','AdaBoost',
                  ada_abs,
                  ada_mae,
                 ada_rmse],index=errors.columns) ,ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

,Группа,Модель,Абсолютная ошибка,MAE,RMSE
0,Слабые игроки,LinearRegression,2.636124e+06,204.41400,256.08000
1,Средние игроки,LinearRegression,2.422902e+06,171.06100,218.08000
2,Мастера,LinearRegression,3.121464e+06,161.26600,206.22300
3,Гроссмейстеры,LinearRegression,2.933576e+05,140.09400,188.90800
4,Общая ошибка,LinearRegression,2.118462e+06,169.20875,217.32275
5,Слабые игроки,AdaBoost,1.053871e+06,272.31800,339.77300
6,Средние игроки,AdaBoost,3.092973e+06,727.75800,912.93700
7,Мастера,AdaBoost,1.130376e+06,194.62400,246.27300
8,Гроссместеры,AdaBoost,1.381863e+05,219.34300,297.87600
9,Общая ошибка,AdaBoost,1.353852e+06,353.51075,449.21475
